In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
Groq_api_key=os.getenv('GROQ_API_KEY')
os.environ['HF_API_KEY']=os.getenv('HF_API_KEY')

In [3]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [4]:
#Wikipedia
wikipedia_api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

In [5]:
wiki.name

'wikipedia'

In [6]:
#Arxiv
arxiv_api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.name

'arxiv'

In [7]:
tools=[wiki,arxiv]

In [8]:
#creating custom rag tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
loader=WebBaseLoader('https://docs.smith.langchain.com/')
docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
Documents=splitter.split_documents(docs)

In [10]:
Embeddings=HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
vectordb=FAISS.from_documents(documents=Documents,embedding=Embeddings)
retriever=vectordb.as_retriever()
retriever

/opt/homebrew/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12f4e2950>, search_kwargs={})

In [11]:
from langchain_classic.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,'langsmith_search','search any information about langsmith')
retriever_tool

Tool(name='langsmith_search', description='search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x12e5a6050>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12f4e2950>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x12f6c1000>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12f4e2950>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', res

In [12]:
retriever_tool.name

'langsmith_search'

In [13]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/homebrew/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x12e5a6050>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12f4e2950>, search_kwargs={}), documen

In [14]:
#Running this tools with agents and llms

In [18]:
from langchain_groq import ChatGroq
import openai

llm_model=ChatGroq(groq_api_key=Groq_api_key,model="openai/gpt-oss-120b")

In [21]:
from langchain_classic import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [26]:
from langchain_classic.agents import create_openai_tools_agent,AgentExecutor
Agent=create_openai_tools_agent(llm_model,tools,prompt)
Agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [27]:
agent_executor=AgentExecutor(agent=Agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [28]:
agent_executor.invoke({'input':'What is langchain'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

**LangChain** is an open‑source software framework designed to make it easier to build applications that incorporate large language models (LLMs) such as OpenAI’s GPT‑4, Anthropic’s Claude, or LLaMA.  

### Core Idea
Instead of treating an LLM as a stand‑alone “black box,” LangChain provides a set of building blocks that let developers **chain together** LLM calls with other components (data sources, prompts, memory, agents, etc.) to create richer, more interactive systems.

### Main Building Blocks
| Block | What it does | Typical use |
|-------|----

{'input': 'What is langchain',
 'output': '**LangChain** is an open‑source software framework designed to make it easier to build applications that incorporate large language models (LLMs) such as OpenAI’s GPT‑4, Anthropic’s Claude, or LLaMA.  \n\n### Core Idea\nInstead of treating an LLM as a stand‑alone “black box,” LangChain provides a set of building blocks that let developers **chain together** LLM calls with other components (data sources, prompts, memory, agents, etc.) to create richer, more interactive systems.\n\n### Main Building Blocks\n| Block | What it does | Typical use |\n|-------|--------------|------------|\n| **Prompt Templates** | Standardizes and parameterizes prompt creation | Re‑using prompts across tasks, injecting variables |\n| **LLM Wrappers** | Uniform API for different LLM providers (OpenAI, Azure, Anthropic, HuggingFace, etc.) | Swap models without changing code |\n| **Chains** | Sequential or parallel composition of LLM calls and other utilities | “Summari

In [29]:
agent_executor.invoke({'input':'What is Agentic Ai'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Agentic AI'}`


Page: Agentic AI
Summary: In the context of generative artificial intelligence, AI agents (also referred to as compound AI systems or agentic AI) are a class of intelligent agents distinguished by their ability to operate autonomously in complex environments. Agentic AI tools prioritize decision-making over content creation and do not require human prompts or continuous oversight.

**Agentic AI** (also called *agentic artificial intelligence*, *AI agents*, or *compound AI systems*) refers to a class of generative‑AI‑driven software agents that are designed to **act autonomously** in complex, often open‑ended environments.  

---

## Core Characteristics

| Feature | What it means for an Agentic AI |
|---------|---------------------------------|
| **Autonomy** | The system can initiate actions, make decisions, and pursue goals without a human issuing a new prompt for each step. |
| **Goal‑ori

{'input': 'What is Agentic Ai',
 'output': '**Agentic AI** (also called *agentic artificial intelligence*, *AI agents*, or *compound AI systems*) refers to a class of generative‑AI‑driven software agents that are designed to **act autonomously** in complex, often open‑ended environments.  \n\n---\n\n## Core Characteristics\n\n| Feature | What it means for an Agentic AI |\n|---------|---------------------------------|\n| **Autonomy** | The system can initiate actions, make decisions, and pursue goals without a human issuing a new prompt for each step. |\n| **Goal‑oriented behavior** | It is equipped with explicit or learned objectives (e.g., “find the cheapest flight”, “optimize a supply‑chain schedule”) and plans a sequence of actions to achieve them. |\n| **Perception‑Action loop** | It continuously observes its environment (via APIs, sensors, web data, etc.), updates an internal state, and selects the next action. |\n| **Tool use** | It can invoke external tools—web browsers, databas

In [30]:
agent_executor.invoke({'input':'Who is Mohanlal'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Mohanlal'}`


Page: Mohanlal
Summary: Mohanlal Viswanathan (IPA: [moːhɐnlaːl ʋiʃʋʷɐn̪aːt̪ʰɐn]; born 21 May 1960), known mononymously as Mohanlal, is an Indian actor and filmmaker who predominantly works in Malayalam cinema and has also occasionally appeared in Tamil, Hindi, Telugu and Kannada films. Mohanlal has a prolific career spanning over four decades, during which he has acted in more than 400 films. The Government of India honoured him with Padma Shri in 2001 and Padma Bhushan in 2019, India's fourth aMohanlal Viswanathan (born 21 May 1960) is a celebrated Indian actor, filmmaker, and occasional playback singer who works primarily in Malayalam cinema, though he has also appeared in Tamil, Hindi, Telugu and Kannada films. Often referred to simply as “Mohanlal,” he began his career in the late 1970s and has since acted in more than 400 movies across a span of over four decades, earning a reputation for

{'input': 'Who is Mohanlal',
 'output': 'Mohanlal\u202fViswanathan (born\u202f21\u202fMay\u202f1960) is a celebrated Indian actor, filmmaker, and occasional playback singer who works primarily in Malayalam cinema, though he has also appeared in Tamil, Hindi, Telugu and Kannada films.\u202fOften referred to simply as “Mohanlal,” he began his career in the late 1970s and has since acted in more than\u202f400\u202fmovies across a span of over four decades, earning a reputation for his versatility, naturalistic style, and ability to portray a wide range of characters—from intense dramatic roles to comic performances.\n\n**Key highlights of his life and career**\n\n| Aspect | Details |\n|--------|---------|\n| **Birthplace** | Elanthoor, Kerala, India |\n| **Debut** | “Thiranottam” (filmed 1978, released 2005) – first released film was “Manjil Virinja Pookkal” (1980) |\n| **Major awards** | 9\u202fNational Film Awards (including Best Actor for *Vanaprastham*, *Bharatham*, *Kaalapani*), 14\u